## Исследование зависимость между динамикой цен эмитентов и их esg рейтингом

In [1]:
full_tickers = ['ADM.L', 'AAL.L', 'ANTO.L', 'AHT.L', 'ABF.L', 'AZN.L', 'AV.L', 'BA.L', 'BARC.L', 'BDEV.L', 'BP.L', 'BATS.L', 'BLND.L', 'BT-A.L', 'BNZL.L', 'BRBY.L', 'CNA.L', 'CCH.L', 'CPG.L', 'CRH.L', 'CRDA.L', 'DCC.L', 'DGE.L', 'EXPN.L', 'FCIT.L', 'FLTR.L', 'FRES.L', 'GLEN.L', 'GSK.L', 'HL.L', 'HSBA.L', 'IHG.L', 'IMB.L', 'INF.L', 'IAG.L', 'ITRK.L', 'KGF.L', 'LAND.L', 'LGEN.L', 'LLOY.L', 'LSEG.L', 'MNDI.L', 'NG.L', 'NWG.L', 'NXT.L', 'PSON.L', 'PSH.L', 'PSN.L', 'PRU.L', 'RKT.L', 'REL.L', 'RTO.L', 'RIO.L', 'RR.L', 'SGE.L', 'SBRY.L', 'SDR.L', 'SMT.L', 'SGRO.L', 'SVT.L', 'SHEL.L', 'SMDS.L', 'SMIN.L', 'SN.L', 'SKG.L', 'SSE.L', 'STAN.L', 'STJ.L', 'TW.L', 'TSCO.L', 'ULVR.L', 'UU.L', 'VOD.L', 'WTB.L', 'WPP.L', 'ALRS.ME', 'CHMF.ME', 'GAZP.ME', 'GMKN.ME', 'HYDR.ME', 'IRAO.ME', 'LKOH.ME', 'MAGN.ME', 'MGNT.ME', 'MTSS.ME', 'NVTK.ME', 'PHOR.ME', 'POLY.ME', 'ROSN.ME', 'SBER.ME', 'SNGS.ME', 'TATN.ME', 'TRNFP.ME', 'VTBR.ME', 'YNDX.ME']

In [218]:
import yfinance as yf
import yesg
import pandas as pd
from nueramic_mathml import HiddenPrints
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
import plotly.express as px
from tqdm import tqdm
from plotly.subplots import make_subplots

def download_data(ticker: str) -> [pd.DataFrame, None]:
    """
    Скачивает ESG рейтинг и значения котировок 
    """
    
    with HiddenPrints():
        esg = yesg.get_historic_esg(ticker)
        if esg is not None:
            start_date = esg.index.min()
            end_date = min(esg.index.max(), pd.Timestamp('2019-11-01'))

            stock = yf.download(ticker, start=start_date, end=end_date, interval='1mo')

            out = pd.merge(esg, stock, right_index=True, left_index=True)
            return out.dropna()

In [219]:
def check_corr_esg_n_adjclose(df: pd.DataFrame) -> pd.DataFrame:
    """
    Проверяет корреляцию между значениями относительных изменений ESG и изменением значений цен закрытия
    """
    
    return df[['Total-Score', 'E-Score', 'S-Score', 'G-Score', 'Adj Close']].corr()


In [220]:
data = pd.DataFrame(
    columns = ['Total-Score', 'E-Score', 'S-Score', 'G-Score', 'Adj Close', 'Linear_R2',
               'linear_w_T', 'linear_w_E', 'linear_w_S', 'linear_w_G']
)

for ticker in tqdm(full_tickers):
    df = download_data(ticker)  # Скачиваем данные 
    if df.shape[0] < 10:
        continue
    _data = check_corr_esg_n_adjclose(df)['Adj Close'].to_list()  # Берем данные о корреляции
    
    x, y = df[['Total-Score', 'E-Score', 'S-Score', 'G-Score']], df['Adj Close']  
    # x = minmax_scale(x)
    # y = minmax_scale(y)
    # Линейная модель
    linear_model = Ridge().fit(x, y)
    # Считаем метрики и смотрим на веса признаков
    _data.append(r2_score(y, linear_model.predict(x)))
    _data.extend(linear_model.coef_)
    
    data.loc[ticker] = _data
    

100%|██████████| 95/95 [01:07<00:00,  1.42it/s]


In [237]:
data.sort_values('Linear_R2', ascending=False).head(10).to_excel('linear.xlsx')
data.sort_values('Linear_R2', ascending=False)

,Total-Score,E-Score,S-Score,G-Score,Adj Close,Linear_R2,linear_w_T,linear_w_E,linear_w_S,linear_w_G
MAGN.ME,0.066009,-0.707369,0.100621,0.733396,1.0,0.952343,-1.956556,-5.622285,-0.838875,0.283090
SBER.ME,-0.185570,-0.927703,0.437700,0.179973,1.0,0.925607,-10.405158,-9.910275,8.756381,-3.040547
YNDX.ME,-0.953390,-0.662200,-0.813202,-0.899404,1.0,0.922984,88.893888,-119.880911,-108.938571,-56.638440
LKOH.ME,0.847254,0.692716,0.940817,0.578414,1.0,0.911741,-5.729748,63.306175,165.370510,-20.830158
CCH.L,-0.823795,-0.664137,-0.911721,-0.495627,1.0,0.890682,-20.356463,14.460455,-104.629750,57.160303
...,...,...,...,...,...,...,...,...,...,...
NG.L,0.092663,-0.041096,0.155303,-0.050959,1.0,0.100915,-5.354806,-10.316413,10.353912,-17.395832
VTBR.ME,0.193068,0.484875,0.164539,0.229198,1.0,0.072430,0.000871,0.001191,-0.000100,0.001489
UU.L,-0.227773,-0.222296,-0.096816,0.079160,1.0,0.071620,-0.872800,-2.268182,-2.642255,3.284623
SNGS.ME,-0.092748,-0.118289,-0.166675,0.131645,1.0,0.071594,3.040533,-1.059241,-1.608418,-0.806390


In [167]:
magn = download_data('MAGN.ME')[['Total-Score', 'E-Score', 'S-Score', 'G-Score', 'Adj Close'] ] 
# magn = pd.DataFrame(minmax_scale(magn), columns=magn.columns, index=magn.index)


In [238]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title_text='<b>Показатели. "Магнитогорский металлургический комбинат"</b>')

fig.add_scatter(y=magn['Adj Close'], x=magn.index, secondary_y=False, name='Adj Close')
fig.add_scatter(y=magn['G-Score'], x=magn.index, secondary_y=True, name='G-Score')
fig.add_scatter(y=magn['E-Score'], x=magn.index, secondary_y=True, name='E-Score', visible='legendonly')
fig.add_scatter(y=magn['S-Score'], x=magn.index, secondary_y=True, name='S-Score', visible='legendonly')
fig.add_scatter(y=magn['Total-Score'], x=magn.index, secondary_y=True, name='Total-Score', visible='legendonly')

fig.update_yaxes(title_text='Цена аквтива', secondary_y=False)
fig.update_yaxes(title_text='Score', secondary_y=True)

In [251]:
from nueramic_mathml.visualize import gen_regression_plot
from nueramic_mathml.ml import LinearRegression
from nueramic_mathml.ml.metrics import r2_score as r2_nueramic
import torch

In [265]:
df = download_data('MAGN.ME')
x_th = torch.tensor(df[['Total-Score', 'E-Score', 'S-Score', 'G-Score']].values)
y_th = torch.tensor(df['Adj Close'].values)
model = LinearRegression().fit(x_th, y_th)
y_th_pred = model(x_th)
r2_nueramic(y_th, y_th_pred)

gen_regression_plot(x_th, y_th, model, title='Линейная регрессия для MAGN.ME')

x_tensor is not 1d. TSNE applied


In [262]:
data[['Total-Score', 'E-Score', 'S-Score', 'G-Score']].abs().sum(axis=1).sort_values(ascending=False).head(10).index

Index(['RIO.L', 'YNDX.ME', 'TATN.ME', 'LKOH.ME', 'GMKN.ME', 'CCH.L', 'AZN.L',
       'CHMF.ME', 'BRBY.L', 'CNA.L'],
      dtype='object')

In [224]:
data.loc[['RIO.L', 'YNDX.ME', 'TATN.ME', 'LKOH.ME', 'GMKN.ME', 'CCH.L', 'AZN.L', 'CHMF.ME', 'BRBY.L', 'CNA.L']].to_excel('slide_14.xlsx')

In [209]:
!ls

README.md                   presentation-f2-03-2011.ppt
catboost_info               slide_14.xlsx
esg-research.ipynb


In [236]:
magn = download_data('YNDX.ME')[['Total-Score', 'E-Score', 'S-Score', 'G-Score', 'Adj Close'] ]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title_text='<b>Показатели. "Яндекс"</b>')

fig.add_scatter(y=magn['Adj Close'], x=magn.index, secondary_y=False, name='Adj Close')
fig.add_scatter(y=magn['G-Score'], x=magn.index, secondary_y=True, name='G-Score')
fig.add_scatter(y=magn['E-Score'], x=magn.index, secondary_y=True, name='E-Score', visible='legendonly')
fig.add_scatter(y=magn['S-Score'], x=magn.index, secondary_y=True, name='S-Score', visible='legendonly')
fig.add_scatter(y=magn['Total-Score'], x=magn.index, secondary_y=True, name='Total-Score', visible='legendonly')

fig.update_yaxes(title_text='Цена аквтива', secondary_y=False)
fig.update_yaxes(title_text='Score', secondary_y=True)

In [235]:
magn = download_data('GMKN.ME')[['Total-Score', 'E-Score', 'S-Score', 'G-Score', 'Adj Close'] ]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title_text='<b>Показатели. "ГМК Норильский Никель"</b>')

fig.add_scatter(y=magn['Adj Close'], x=magn.index, secondary_y=False, name='Adj Close')
fig.add_scatter(y=magn['G-Score'], x=magn.index, secondary_y=True, name='G-Score')
fig.add_scatter(y=magn['E-Score'], x=magn.index, secondary_y=True, name='E-Score', visible='legendonly')
fig.add_scatter(y=magn['S-Score'], x=magn.index, secondary_y=True, name='S-Score', visible='legendonly')
fig.add_scatter(y=magn['Total-Score'], x=magn.index, secondary_y=True, name='Total-Score', visible='legendonly')

fig.update_yaxes(title_text='Цена аквтива', secondary_y=False)
fig.update_yaxes(title_text='Score', secondary_y=True)

In [234]:
magn = download_data('LKOH.ME')[['Total-Score', 'E-Score', 'S-Score', 'G-Score', 'Adj Close'] ]


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title_text='<b>Показатели. "ЛУКОЙЛ"</b>')

fig.add_scatter(y=magn['Adj Close'], x=magn.index, secondary_y=False, name='Adj Close')
fig.add_scatter(y=magn['G-Score'], x=magn.index, secondary_y=True, name='G-Score', visible='legendonly')
fig.add_scatter(y=magn['E-Score'], x=magn.index, secondary_y=True, name='E-Score', visible='legendonly')
fig.add_scatter(y=magn['S-Score'], x=magn.index, secondary_y=True, name='S-Score')
fig.add_scatter(y=magn['Total-Score'], x=magn.index, secondary_y=True, name='Total-Score', visible='legendonly')

fig.update_yaxes(title_text='Цена аквтива', secondary_y=False)
fig.update_yaxes(title_text='Score', secondary_y=True)